In [50]:
from ising_model import ClassicIsing
import electron
from grid import Grid, HoleGrid, Mobius, Cylinder, Torus
import matplotlib.pyplot as plt
import numpy as np

In [51]:

def B_vs_T(ising_type, grid_type, temps, eq_steps, mc_steps, runs_per_T):
    average_mag = []
    for T in temps:
        total_mag = 0
        for run in range(runs_per_T):
        
            newGrid = grid_type(n_x=15, n_y=15, gridPointObject=electron.ClassicElectron, random_seed=12)
            isingModel = ising_type(newGrid, temperature=T, ferromagnetivity= 1.7, Mf_External=0)
            isingModel.runSimulation(eq_steps)
            mag = isingModel.magnetization()
            total_mag += mag

        print(f"T={T:.2f}, <|M|>={total_mag/runs_per_T:.3f}")

        average_mag.append(total_mag/runs_per_T)


            

    return average_mag


In [ ]:
# --- Parameters ---
N = 15            # Lattice size
J = 1          # Coupling constant
temps = np.linspace(1, 3.5, 30)  # Temperature range
eq_steps = 30  # Steps for equilibration
mc_steps = 25  # Steps for measurement
runs_per_T = 30    # Average over multiple runs



mags = B_vs_T(ClassicIsing, Grid, temps, eq_steps, mc_steps, runs_per_T)


# graphing the results
plt.plot(temps, mags, marker='o')
plt.xlabel('Temperature (T)')
plt.ylabel('Average Magnetization |M|')
plt.title('Magnetization vs Temperature for normal grid')
plt.text(0.7, 0.9, f'N={N}, J={J}', transform=plt.gca().transAxes)
plt.legend()
plt.grid()
plt.show()

T=0.10, <|M|>=0.067
T=0.22, <|M|>=0.067
T=0.33, <|M|>=0.067
T=0.45, <|M|>=0.067
T=0.57, <|M|>=0.076
T=0.69, <|M|>=0.111
T=0.80, <|M|>=1.000
T=0.92, <|M|>=0.991
T=1.04, <|M|>=0.991
T=1.16, <|M|>=0.102
T=1.27, <|M|>=0.973
T=1.39, <|M|>=0.991
T=1.51, <|M|>=0.360
T=1.62, <|M|>=0.991
T=1.74, <|M|>=0.849
T=1.86, <|M|>=0.964
T=1.98, <|M|>=0.876
T=2.09, <|M|>=0.804
T=2.21, <|M|>=0.911
T=2.33, <|M|>=0.191


KeyboardInterrupt: 